In [2]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import os
import warnings

# 1. Importar Normalización Ganadora
from normalization_functions import pipeline_d_normalize
# 2. Importar Características Aumentadas 
from lexicon_functions import get_augmented_features

In [3]:

# --- 1. Configuración de Rutas ---
TRAIN_PATH = os.path.join("..", "data", "processed", "train.csv")
RESULTS_PATH = os.path.join("..", "results", "fase_3_mlp_results.csv")

os.makedirs(os.path.dirname(RESULTS_PATH), exist_ok=True)
warnings.filterwarnings("ignore", category=UserWarning)

# --- 2. Cargar y Preparar Datos ---
print(f"Iniciando Fase 3: MLP + GridSearchCV")
try:
    df_train = pd.read_csv(TRAIN_PATH)
    print(f"Datos de entrenamiento cargados desde '{TRAIN_PATH}'.")
except FileNotFoundError:
    print(f"\n¡Error de Archivo! No se encontró '{TRAIN_PATH}'.")
    exit()

print("Aplicando 'pipeline_d_normalize' (Combinación Ganadora) a los datos...")
# Aplicar la normalización GANADORA
X_train = df_train['text'].apply(pipeline_d_normalize)
y_train = df_train['Polarity']
print(f"Datos normalizados listos. {len(X_train)} documentos.")

# --- 3. Definir Pipeline y Rejilla de Búsqueda ---

# Crear el Pipeline con la combinación GANADORA
text_pipeline = Pipeline([
    ('features', get_augmented_features()), 
    ('classifier', MLPClassifier(random_state=0, max_iter=500))
])

# Definir la rejilla de hiperparámetros para MLP
# 'classifier__' es el prefijo para aplicar los parámetros al paso 'classifier' del pipeline
param_grid = {
    # Probar una capa oculta de 50 neuronas vs. una de 100
    'classifier__hidden_layer_sizes': [(50,), (100,)],
    
    # Probar dos valores de regularización (alpha)
    'classifier__alpha': [0.001, 0.01],
    
    # 'relu' es casi siempre la mejor opción
    'classifier__activation': ['relu']
}

print(f"Rejilla de Hiperparámetros (Grid): {param_grid}")
print(f"Total de combinaciones a probar: {len(param_grid['classifier__hidden_layer_sizes']) * len(param_grid['classifier__alpha'])} = 4")
print("Cada combinación se entrenará con 5-fold CV (Total: 20 entrenamientos de MLP)")


Iniciando Fase 3: MLP + GridSearchCV
Datos de entrenamiento cargados desde '..\data\processed\train.csv'.
Aplicando 'pipeline_d_normalize' (Combinación Ganadora) a los datos...
Datos normalizados listos. 24169 documentos.
Rejilla de Hiperparámetros (Grid): {'classifier__hidden_layer_sizes': [(50,), (100,)], 'classifier__alpha': [0.001, 0.01], 'classifier__activation': ['relu']}
Total de combinaciones a probar: 4 = 4
Cada combinación se entrenará con 5-fold CV (Total: 20 entrenamientos de MLP)


In [ ]:

# --- 4. Configurar y Ejecutar GridSearchCV ---

# verbose=2 para que puedas ver el progreso y no pienses que se trabó
grid_search = GridSearchCV(
    estimator=text_pipeline,
    param_grid=param_grid,
    cv=5,                # 5 folds (como en la especificación)
    scoring='f1_macro',  # La métrica que estamos usando
    n_jobs=-1,           # Usar todos los núcleos de CPU
    verbose=2            
)

print("\n--- ¡INICIANDO GRIDSEARCHCV! ---")
print("Esto puede tardar varios minutos...")
try:
    grid_search.fit(X_train, y_train)
except Exception as e:
    print(f"\n¡Error durante el entrenamiento de MLP! {e}")
    exit()

print("\n--- ¡GridSearchCV completado! ---")

# --- 5. Guardar y Reportar Resultados ---

best_score = grid_search.best_score_
best_params = grid_search.best_params_

print(f"Mejor F1-Macro (promedio de CV): {best_score:.4f}")
print(f"Mejores Hiperparámetros encontrados: {best_params}")

# Guardar los resultados
results_df = pd.DataFrame({
    "model": "MLPClassifier",
    "pipeline_base": "D (Negación) + Frequency",
    "best_avg_f1_macro": [best_score],
    "best_params": [str(best_params)] # Guardar como string
})

results_df.to_csv(RESULTS_PATH, index=False)
print(f"Resultados de la Fase 3 guardados en '{RESULTS_PATH}'")


--- ¡INICIANDO GRIDSEARCHCV! ---
Esto puede tardar varios minutos...
Fitting 5 folds for each of 4 candidates, totalling 20 fits

--- ¡GridSearchCV completado! ---
Mejor F1-Macro (promedio de CV): 0.4341
Mejores Hiperparámetros encontrados: {'classifier__activation': 'relu', 'classifier__alpha': 0.01, 'classifier__hidden_layer_sizes': (50,)}
Resultados de la Fase 3 guardados en '..\results\fase_3_mlp_results.csv'
